In [29]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
import re
import pytest


headers = {'user-agent': 'Mozilla/5.0'}

#lista unificada para cada atributo html
list_todos = []

In [30]:
"""
Esta função faz uma requisição GET para cada página de resultados da busca por "nintendo switch" no site da Magazine Luiza,
extrai várias informações sobre cada produto listado e as salva em um arquivo JSON.
"""
def get_data():

    for page_number in range(1,17):
        # Faça uma requisição GET para cada página de resultados
        resposta = requests.get(f"https://www.magazineluiza.com.br/busca/nintendo+switch/?page={page_number}")
        sopa = resposta.text
        sopa_bonita = BeautifulSoup(sopa,'html.parser')

        # Encontre todos os elementos relevantes na página
        list_titulo = sopa_bonita.find_all('h2',{'data-testid':'product-title'})
        list_preco_promo = sopa_bonita.find_all('p',{'data-testid':'price-value'})
        list_condition_promo = sopa_bonita.find_all('span',{'data-testid':'in-cash'})
        list_parcelamento = sopa_bonita.find_all('p',{'data-testid':'installment'})
        img_tags = sopa_bonita.find_all('img')
        img_srcs = [img['src'] for img in img_tags]

        for titulo, preco_promo, condition_promo, parcelado, img in zip(list_titulo, list_preco_promo, list_condition_promo, list_parcelamento, img_srcs):
            # Extraia o texto de cada elemento e faça algumas limpezas
            titulo = titulo.text
            moeda = preco_promo.text
            moeda = moeda[0] + moeda[1]
            preco_promo = preco_promo.text.replace('R$','').replace('\xa0','').replace('.','').replace(',','.')
            condition_promo = condition_promo.text
            parcelado = parcelado.text.replace('R$','').replace('\xa0','').replace('.','').replace(',','.')
            imagem = str(img).replace('[', '').replace(']', '')

            # Adicione os atributos à lista 'list_todos' como um dicionário
            list_todos.append({
                'titulo': titulo,
                'moeda': moeda,
                'preco_promo': preco_promo,
                'condition_promo': condition_promo,
                'parcelado': parcelado,
                'imagem': imagem
            })

    # Transforme a lista 'list_todos' em JSON e salve em um arquivo
    with open('list_todos.json', 'w') as f:
        json.dump(list_todos, f)

    return resposta.status_code, list_todos

# Chame a função get_data() e obtenha o status do request e a lista 'list_todos'
status_code, list_todos = get_data()


In [31]:
# Verifique se o status do request é 200
assert status_code == 200, "O status do request não é 200"
# Verifique se a lista 'list_todos' não está vazia
assert list_todos, "A lista 'list_todos' está vazia"


In [ ]:
df = pd.DataFrame(list_todos)


In [33]:
df['preco_promo'] = df['preco_promo'].astype(np.float64).round(2)

In [34]:
def extrair_memoria(info):
        # Usar regex para encontrar padrões "GB" ou "Gb" ou "gb" seguidos por números
    padrao = r'(\d+)\s*(G[gBb])'
    resultado = re.search(padrao, info)
    if resultado:
        # Se encontrado, retornar a correspondência
        return resultado.group(0)
    else:
        # Se não encontrado, retornar uma string vazia
        return '-'

# Aplicar a função e criar a nova coluna "Memoria"
df['memoria'] = df['titulo'].apply(extrair_memoria)

In [35]:
oled = df['titulo'].str.contains('Oled', case=False)
oled = oled.replace({True: 'Sim', False: 'Nao'})
df['oled'] = oled

lite = df['titulo'].str.contains('Lite', case=False)
lite = lite.replace({True: 'Sim', False: 'Nao'})
df['lite'] = lite

controle = df['titulo'].str.contains('Joy-con', case=False)
controle = controle.replace({True: 'Sim', False: 'Nao'})
df['joy_con'] = controle

In [36]:
df

,titulo,moeda,preco_promo,condition_promo,parcelado,imagem,memoria,oled,lite,joy_con
0,Console Nintendo Switch Oled 64GB Cinza Edição...,R$,2314.45,no Pix,ou 2386.03 em 10x de 238.60 sem juros,https://a-static.mlcdn.com.br/280x210/console-...,64GB,Sim,Nao,Nao
1,Nintendo Switch 64GB 2 Controles Joy-con,R$,2564.05,no Pix,ou 2699.00 em 10x de 269.90 sem juros,https://a-static.mlcdn.com.br/280x210/nintendo...,64GB,Nao,Nao,Sim
2,Nintendo Switch Lite Turquesa 32GB - HDHSBAZA1BRA,R$,1142.10,no Pix,ou 1269.00 em 10x de 126.90 sem juros,https://a-static.mlcdn.com.br/280x210/nintendo...,32GB,Nao,Sim,Nao
3,Console Nintendo Switch Mario Kart 8 Digital D...,R$,2085.50,no Pix,ou 2150.00 em 10x de 215.00 sem juros,https://a-static.mlcdn.com.br/280x210/console-...,-,Nao,Nao,Sim
4,"Nintendo Switch Lite 32GB Coral 5,5”",R$,1648.03,no Pix,ou 1699.00 em 10x de 169.90 sem juros,https://i.mlcdn.com.br/selo-ml/65x50/e412ff6a-...,32GB,Nao,Sim,Nao
...,...,...,...,...,...,...,...,...,...,...
336,Película De Vidro Temperado Compatível com Nin...,R$,33.95,no Pix,ou 376.80 em 6x de 62.80 sem juros,https://a-static.mlcdn.com.br/280x210/pelicula...,-,Sim,Nao,Nao
337,Dokapon Kingdom: Connect - Switch,R$,332.18,no Pix,ou 309.80 em 6x de 51.63 sem juros,https://a-static.mlcdn.com.br/280x210/dokapon-...,-,Nao,Nao,Nao
338,Case Compatível Nintendo Switch Lite Bolsa Est...,R$,39.99,no Pix,ou 376.80 em 6x de 62.80 sem juros,https://a-static.mlcdn.com.br/280x210/case-com...,-,Nao,Sim,Nao
339,Case Compatível Nintendo Switch Lite Bolsa Est...,R$,39.99,no Pix,ou 252.90 em 5x de 50.58 sem juros,https://a-static.mlcdn.com.br/280x210/case-com...,-,Nao,Sim,Nao
